Resources

- [SAPIEN — sapien 3.0 documentation](https://sapien-sim.github.io/docs/index.html)
- [Transforms3d](https://matthew-brett.github.io/transforms3d/index.html)

In [1]:
from sapien import Pose, Scene
from transforms3d.euler import euler2quat
from PIL import Image
import numpy as np

from loguru import logger

from pathlib import Path
from shutil import rmtree

/home/ajcd2020/.local/lib/python3.9/site-packages/sapien/_vulkan_tricks.py:37: UserWarning: Failed to find Vulkan ICD file. This is probably due to an incorrect or partial installation of the NVIDIA driver. SAPIEN will attempt to provide an ICD file anyway but it may not work.
  warn(


# Joint Control

```
force = stiffness * (target_position - position) + damping * (target_velocity - velocity)
```

A built-in PD controller is supported by revolute and prismatic joints. You can use set_drive_properties to specify the parameters for this controller. The following properties are associated with the PD controller.

| Property    | Type  | Description                                       |
| ----------- | ----- | ------------------------------------------------- |
| stiffness   | float | Stiffness (Kp) for the PD drive                   |
| damping     | float | Damping (Kd) for the PD drive                     |
| force_limit | float | Maximum force that can be applied by the PD drive |
| drive_mode  | str   | One of “force” or “acceleration”                  |

In acceleration mode, the PD drive becomes invariant to link masses by controlling acceleration directly.

In [2]:
def compute_camera_pose_follow(target, offset):
    "Compute the camera pose by specifying forward(x), left(y) and up(z)."
    
    position = np.array(offset)
    
    forward = np.array(target) - position
    forward = forward / np.linalg.norm(forward)

    left = np.cross([0, 0, 1], forward)
    left = left / np.linalg.norm(left)
    
    up = np.cross(forward, left)
    
    matrix = np.eye(4)
    matrix[:3, :3] = np.stack([forward, left, up], axis=1)
    matrix[:3, 3] = position

    return Pose(matrix)

In [10]:
# TODO:
# - density
# - configure lighting instead of hardcode
# - damping on motors
# - base wheel_extension_angle_offset on number of extensions

timestep = 1 / 100.0

static_friction = 10.0
dynamic_friction = 10.0
restitution = 0.1
joint_friction = 0.0
joint_damping = 0.0

# The coordinate frame in Sapien is: x(forward), y(left), z(upward)

chassis_length = 16e-2
chassis_width = 32e-2
chassis_thickness = 3e-2
chassis_material = [0.4, 0.4, 0.0]

wheel_radius = 5e-2
wheel_thickness = 1e-2
wheel_material = [0.0, 0.4, 0.4]

num_wheel_extensions = 3
wheel_extension_radial_offset = wheel_radius / 1.2
wheel_extension_angle_offset = np.deg2rad(20)
wheel_extension_length = wheel_radius / 3
wheel_extension_width = wheel_thickness
wheel_extension_thickness = wheel_thickness
wheel_extension_material = [0.4, 0.0, 0.4]


def simulate(on_blocks=False):
    
    # 
    # Scene
    # 
    
    scene = Scene()
    
    scene.set_timestep(timestep)
    
    scene.set_ambient_light([0.5, 0.5, 0.5])
    scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5])
    
    scene.add_ground(altitude=0)
    
    # 
    # Blocks
    # 

    if on_blocks:
        half_size = [4e-2, 4e-2, 4e-2]
        builder = scene.create_actor_builder()
        builder.add_box_collision(half_size=half_size)
        builder.add_box_visual(half_size=half_size, material=[.1, .1, .1])
        box = builder.build(name="block")

    # 
    # Chassis
    # 
    
    robot_builder = scene.create_articulation_builder()
    
    chassis_half_size = [chassis_length / 2, chassis_width / 2, chassis_thickness / 2]
    chassis_vertical_offset = wheel_radius + 7e-2
    chassis_pose = Pose(p=[0, 0, chassis_vertical_offset])
    
    chassis = robot_builder.create_link_builder()
    chassis.set_name("chassis")
    chassis.add_box_collision(half_size=chassis_half_size)
    chassis.add_box_visual(half_size=chassis_half_size, material=chassis_material)
    
    # 
    # Wheels and revolute joints
    # 
    
    wheel_half_thickness = wheel_thickness / 2
    
    front_rear_placement = chassis_half_size[0]
    left_right_placement = chassis_half_size[1] + 1e-2
    ninety_deg = np.deg2rad(90)
    
    wheel_parameters = [
        ("front_left",  front_rear_placement,  left_right_placement, euler2quat(0, 0, ninety_deg)),
        ("front_right", front_rear_placement, -left_right_placement, euler2quat(0, 0, ninety_deg)),
        ("rear_left",  -front_rear_placement,  left_right_placement, euler2quat(0, 0, ninety_deg)),
        ("rear_right", -front_rear_placement, -left_right_placement, euler2quat(0, 0, ninety_deg)),
    ]
    
    wheels = {}
    
    for name, fr, lr, quat in wheel_parameters:
        
        wheel = robot_builder.create_link_builder(chassis)
        wheel.set_name(f"wheel_{name}")
        wheel.set_joint_name(f"wheel_joint_{name}")
    
        # TODO: convert to spheroid using convex mesh?
        # NOTE: by default, cylinders are oriented along the x-axis
        wheel.add_cylinder_collision(radius=wheel_radius, half_length=wheel_half_thickness)
        wheel.add_cylinder_visual(radius=wheel_radius, half_length=wheel_half_thickness, material=wheel_material)
    
        # wheel_half_size = [wheel_thickness/2, wheel_radius, wheel_radius]
        # wheel.add_box_collision(half_size=wheel_half_size)
        # wheel.add_box_visual(half_size=wheel_half_size, material=wheel_material)
    
        wheel.set_joint_properties(
            "revolute",
            limits=[[-np.inf, np.inf]],
            pose_in_parent=Pose(p=[fr, lr, 0], q=quat),
            pose_in_child=Pose(),
            friction=joint_friction,
            damping=joint_damping,
        )
    
        wheels[name] = wheel
    
    # 
    # Wheel extensions
    # 
    
    # NOTE: extensions are relative to the wheel:
    #    x -> y
    #    y -> x
    #    z -> z
    
    extension_half_size = [wheel_extension_width / 2, wheel_extension_length / 2, wheel_extension_thickness / 2]
    
    for name, fr, lr, quat in wheel_parameters:
        for i in range(num_wheel_extensions):
        
            extension = robot_builder.create_link_builder(wheels[name])
            extension.set_name(f"extension_{name}_{i}")
            extension.set_joint_name(f"extension_joint_{name}_{i}")
    
            # TODO: convert to capsule?
            extension.add_box_collision(half_size=extension_half_size)
            extension.add_box_visual(half_size=extension_half_size, material=wheel_extension_material)
    
            radial_angle = np.deg2rad(i/num_wheel_extensions*360)
            
            y = wheel_extension_radial_offset * np.cos(radial_angle)
            z = wheel_extension_radial_offset * np.sin(radial_angle)
    
            x = np.copysign(wheel_extension_width, lr)
    
            extension.set_joint_properties(
                "revolute",
                # TODO: Upper limit should take into account angle offset
                limits=[[0, np.pi]],
                pose_in_parent=Pose(p=[x, y, z]),
                pose_in_child=Pose(p=[0, -wheel_extension_length/2, 0], q=euler2quat(np.deg2rad(90) - radial_angle + wheel_extension_angle_offset, 0, 0)),
                friction=joint_friction,
                damping=joint_damping,
            )
    
    # 
    # Finalize the articulated robot
    # 
    
    robot = robot_builder.build()
    robot.set_name("robot")
    robot.set_pose(chassis_pose)
    
    joints = {joint.get_name(): joint for joint in robot.get_active_joints()}
    
    joint_mode = 'force'
    
    for jname in joints:
        
        if jname.startswith("wheel_joint"):
            joints[jname].set_drive_properties(stiffness=10, damping=100, mode=joint_mode)
        
        elif jname.startswith("extension_joint"):
            joints[jname].set_drive_properties(stiffness=1000, damping=10, mode=joint_mode)
        
        else:
            print("Ignoring", jname)
    
    
    # 
    # Camera
    #

    # camera_target = [0, 0, 0]
    camera_target = [0.5, 0, 0]

    camera_offset = [-0.2, -0.6, 1]
    # camera_offset = [-0.6, -0.6, 0.8]  # default
    # camera_offset = [-1.5, -0.6, 2.4]  # fixed trailing view
    # camera_offset = [ 0.0, -0.6, 0.2]  # from right
    # camera_offset = [ 0.0,  0.6, 0.1]  # from left
    # camera_offset = [-0.1,  0.0, 1.0]  # from top
    # camera_offset = [-0.8,  0.0, 0.5]  # from back
    
    camera_pose = compute_camera_pose_follow(camera_target, camera_offset)
    
    near, far = 0.1, 100
    width, height = 640, 480
    fov = np.deg2rad(35)
    
    camera = scene.add_camera(
        name="camera",
        width=width,
        height=height,
        fovy=fov,
        near=near,
        far=far,
    )
    camera.set_entity_pose(camera_pose)
    
    # camera = scene.add_mounted_camera(
    #     name="mounted_camera",
    #     mount=robot.find_link_by_name("chassis").entity,
    #     pose=camera_pose,
    #     width=width,
    #     height=height,
    #     fovy=fov,
    #     near=near,
    #     far=far,
    # )
    
    
    # 
    # Simulate
    #
    
    image_dir = Path("./image_output")
    
    if image_dir.exists():
        rmtree(image_dir)
    
    image_dir.mkdir()
    
    duration = 8
    total_steps = int(duration / timestep)
    
    capture_fps = 25
    capture_interval = int((1/capture_fps) / timestep)
    capture_i = 0
    
    progress_interval = 200
    
    wheel_speed = 3
    extension_position = 0

    for jname in joints:
        
        if jname.startswith("wheel_joint"):
            joints[jname].set_drive_velocity_target(wheel_speed)
        
        elif jname.startswith("extension_joint"):
            joints[jname].set_drive_target(extension_position)
    
    for step in range(total_steps):
    
        # NOTE: update joints here

        if step > total_steps // 2:
            extension_position = np.deg2rad(90)
    
            for jname in joints:                
                if jname.startswith("extension_joint"):
                    joints[jname].set_drive_target(extension_position)
    
        robot.set_qf(robot.compute_passive_force(gravity=True, coriolis_and_centrifugal=True))
    
        scene.step()
        scene.update_render()
    
        if step % 500 == 0 or step == (total_steps - 1):
            logger.debug('step:', step)
            logger.debug('Pose', robot.get_pose())
            logger.debug('Joint positions', robot.get_qpos())
            logger.debug('Joint velocities', robot.get_qvel())
    
        if step % capture_interval == 0:       
            camera.take_picture()
            rgba = camera.get_picture("Color")
            rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
            rgba_pil = Image.fromarray(rgba_img)
            rgba_pil.save(f"image_output/cam{capture_i:05}.png")
            capture_i += 1
    
    logger.debug("Done")

simulate()

2025-01-23 22:21:37.274 | DEBUG    | __main__:simulate:275 - step:
2025-01-23 22:21:37.275 | DEBUG    | __main__:simulate:276 - Pose
2025-01-23 22:21:37.275 | DEBUG    | __main__:simulate:277 - Joint positions
2025-01-23 22:21:37.276 | DEBUG    | __main__:simulate:278 - Joint velocities
2025-01-23 22:21:39.468 | DEBUG    | __main__:simulate:275 - step:
2025-01-23 22:21:39.470 | DEBUG    | __main__:simulate:276 - Pose
2025-01-23 22:21:39.470 | DEBUG    | __main__:simulate:277 - Joint positions
2025-01-23 22:21:39.471 | DEBUG    | __main__:simulate:278 - Joint velocities
2025-01-23 22:21:40.652 | DEBUG    | __main__:simulate:275 - step:
2025-01-23 22:21:40.653 | DEBUG    | __main__:simulate:276 - Pose
2025-01-23 22:21:40.653 | DEBUG    | __main__:simulate:277 - Joint positions
2025-01-23 22:21:40.654 | DEBUG    | __main__:simulate:278 - Joint velocities
2025-01-23 22:21:40.654 | DEBUG    | __main__:simulate:288 - Done


In [11]:
!/class/cs152/miniforge3/envs/sapien/bin/ffmpeg -y -framerate 25 -i image_output/cam%05d.png -pix_fmt yuv420p test.mp4

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/class/cs152/miniforge3/envs/sapien --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-li